# Multi-Agent ETL Copilot for Tabular Data (Kaggle Agents Capstone)

This notebook implements a **multi-agent ETL Copilot** using the Google Agent Development Kit (ADK).

The system:

- Accepts a **source tabular dataset** (example: nurse staffing data).
- Extracts and summarizes the **source schema**.
- Maps the source schema to a **target analytics schema**.
- Generates **pandas transformation code**.
- Validates the code with a **ValidationAgent**.
- Generates human-readable **Markdown documentation** with a **DocumentationAgent**.

Although the example dataset is nurse staffing, the agents and prompts are designed to be **generic** and reusable for arbitrary tabular datasets (e.g., finance, operations, customer data).


## 🟦 Step 1 — Load Required Libraries
In this step, I load the essential Python libraries used for data loading and exploration.

- **pandas** for reading CSV files  
- **numpy** for basic numerical operations  
- additional libraries will be introduced when needed

This prepares the notebook environment for analyzing the nurse staffing dataset.


## 🟦 Step 2 — Load the Nurse Staffing Dataset
I load the nurse staffing CSV file provided in the Kaggle dataset.

Kaggle datasets are usually stored in:
/kaggle/input

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nurse-staff-csv/PBJ_Daily_Nurse_Staffing_Q2_2024.csv


Once loaded, I preview the first few rows to understand its structure and content.
## 🟦 Step 3 — Preview the Dataset
Before building any agents, I preview the dataset using:

- `df.head()` for sample rows  
- `df.shape` for number of rows and columns  
- `df.describe()` for basic numeric summaries  

This helps me confirm:
- the dataset loaded correctly  
- the column naming patterns  
- numeric vs categorical fields  


In [4]:
import pandas as pd

df = pd.read_csv('/kaggle/input/nurse-staff-csv/PBJ_Daily_Nurse_Staffing_Q2_2024.csv', encoding='latin1')
df.head()

/tmp/ipykernel_47/820383754.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/nurse-staff-csv/PBJ_Daily_Nurse_Staffing_Q2_2024.csv', encoding='latin1')


,PROVNUM,PROVNAME,CITY,STATE,COUNTY_NAME,COUNTY_FIPS,CY_Qtr,WorkDate,MDScensus,Hrs_RNDON,...,Hrs_LPN_ctr,Hrs_CNA,Hrs_CNA_emp,Hrs_CNA_ctr,Hrs_NAtrn,Hrs_NAtrn_emp,Hrs_NAtrn_ctr,Hrs_MedAide,Hrs_MedAide_emp,Hrs_MedAide_ctr
0,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240401,51,10.77,...,0.0,160.08,160.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240402,52,8.43,...,0.0,135.95,135.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240403,53,11.13,...,0.0,150.31,150.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240404,52,12.27,...,0.0,133.01,133.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240405,52,4.95,...,0.0,137.92,137.92,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 🟦 Step 4 — Extract the Source Schema
To build an ETL Copilot, the first agent (SchemaAgent) must understand the dataset structure.

In this step, I extract:
- column names  
- data types  
- non-null counts  

I use `df.info()` for a quick view, and then a small helper tool to convert the schema into a structured Python list.

This is the first building block for the **SchemaAgent**.
 


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1325324 entries, 0 to 1325323
Data columns (total 33 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   PROVNUM           1325324 non-null  object 
 1   PROVNAME          1325324 non-null  object 
 2   CITY              1325324 non-null  object 
 3   STATE             1325324 non-null  object 
 4   COUNTY_NAME       1325324 non-null  object 
 5   COUNTY_FIPS       1325324 non-null  int64  
 6   CY_Qtr            1325324 non-null  object 
 7   WorkDate          1325324 non-null  int64  
 8   MDScensus         1325324 non-null  int64  
 9   Hrs_RNDON         1325324 non-null  float64
 10  Hrs_RNDON_emp     1325324 non-null  float64
 11  Hrs_RNDON_ctr     1325324 non-null  float64
 12  Hrs_RNadmin       1325324 non-null  float64
 13  Hrs_RNadmin_emp   1325324 non-null  float64
 14  Hrs_RNadmin_ctr   1325324 non-null  float64
 15  Hrs_RN            1325324 non-null  float64
 16  

In [7]:
# Helper: custom "schema extraction tool" for agents

def extract_schema(df: pd.DataFrame):
    """
    Convert df.dtypes into a structured list of dicts.
    This acts as a simple custom tool for the future SchemaAgent.
    """
    schema = []
    for col, dtype in df.dtypes.items():
        schema.append({
            "column": col,
            "dtype": str(dtype)
        })
    return schema

source_schema = extract_schema(df)
source_schema[:10]


[{'column': 'PROVNUM', 'dtype': 'object'},
 {'column': 'PROVNAME', 'dtype': 'object'},
 {'column': 'CITY', 'dtype': 'object'},
 {'column': 'STATE', 'dtype': 'object'},
 {'column': 'COUNTY_NAME', 'dtype': 'object'},
 {'column': 'COUNTY_FIPS', 'dtype': 'int64'},
 {'column': 'CY_Qtr', 'dtype': 'object'},
 {'column': 'WorkDate', 'dtype': 'int64'},
 {'column': 'MDScensus', 'dtype': 'int64'},
 {'column': 'Hrs_RNDON', 'dtype': 'float64'}]

## 🟦 Step 5 — Convert Schema Into Prompt-Friendly Format

Large raw outputs (like `df.info()`) are not ideal for AI models.

So I perform **context engineering** by converting the schema into a compact bullet-list format:

- `- column_name: dtype`

This makes it easier for an LLM-based agent to reason about mappings.

This is how I prepare the **context** for the Mapping Agent.


In [8]:
# Step 5 - Build a bullet-list version of the schema for LLM context

def schema_to_bullets(schema):
    """
    Turn the schema list into a bullet-list string:
    - COLUMN: dtype
    """
    lines = [f"- {col['column']}: {col['dtype']}" for col in schema]
    return "\n".join(lines)

source_schema_text = schema_to_bullets(source_schema)
print(source_schema_text)


- PROVNUM: object
- PROVNAME: object
- CITY: object
- STATE: object
- COUNTY_NAME: object
- COUNTY_FIPS: int64
- CY_Qtr: object
- WorkDate: int64
- MDScensus: int64
- Hrs_RNDON: float64
- Hrs_RNDON_emp: float64
- Hrs_RNDON_ctr: float64
- Hrs_RNadmin: float64
- Hrs_RNadmin_emp: float64
- Hrs_RNadmin_ctr: float64
- Hrs_RN: float64
- Hrs_RN_emp: float64
- Hrs_RN_ctr: float64
- Hrs_LPNadmin: float64
- Hrs_LPNadmin_emp: float64
- Hrs_LPNadmin_ctr: float64
- Hrs_LPN: float64
- Hrs_LPN_emp: float64
- Hrs_LPN_ctr: float64
- Hrs_CNA: float64
- Hrs_CNA_emp: float64
- Hrs_CNA_ctr: float64
- Hrs_NAtrn: float64
- Hrs_NAtrn_emp: float64
- Hrs_NAtrn_ctr: float64
- Hrs_MedAide: float64
- Hrs_MedAide_emp: float64
- Hrs_MedAide_ctr: float64


## 🟦 Step 6 — Define the Target Schema (Final Structure)

To drive ETL mapping, I define a **target schema**:  
the final cleaned structure I want the nurse staffing data to be transformed into.

### 📌 Target Schema

```text
facility_id
facility_name
state
county_name
quarter
work_date
resident_census
rn_hours_total
lpn_hours_total
cna_hours_total
total_nursing_hours


Field Definitions

facility_id → unique facility identifier

facility_name → name of the facility

state → U.S. state

county_name → county name

quarter → quarter-year period (from CY_Qtr)

work_date → report date (from WorkDate)

resident_census → number of residents (MDScensus)

rn_hours_total → combined RN-related hours

lpn_hours_total → combined LPN-related hours

cna_hours_total → combined CNA / aide hours

total_nursing_hours → RN + LPN + CNA totals

This target schema is the goal for the Mapping Agent.

In [9]:

# Step 6 - Represent target schema in code (for use in prompts / agents)

target_schema_fields = [
    "facility_id",
    "facility_name",
    "state",
    "county_name",
    "quarter",
    "work_date",
    "resident_census",
    "rn_hours_total",
    "lpn_hours_total",
    "cna_hours_total",
    "total_nursing_hours",
]

target_schema_text = "\n".join(target_schema_fields)
print(target_schema_text)


facility_id
facility_name
state
county_name
quarter
work_date
resident_census
rn_hours_total
lpn_hours_total
cna_hours_total
total_nursing_hours


## 🟦 Step 7 — Build the Mapping Prompt (Source → Target)

Now I construct a **Mapping Prompt** that will be sent to the AI model (Gemini).

This prompt gives the model:

1. The **source schema** (bullet-list form)
2. The **target schema** (final structure)
3. Clear instructions to:
   - map each target field to the best source column
   - indicate when multiple columns should be combined (e.g., RN + LPN + CNA hours)
   - flag any missing matches
   - explain its reasoning

This prompt powers the **MappingAgent** in the ETL Copilot.


In [11]:
# Step 7 - Build the mapping prompt text

mapping_prompt = f"""
You are a Data Mapping Agent.

Here is the source schema extracted from the dataset:
{source_schema_text}

Here is the target schema I want to map to:
{target_schema_text}

Your task:
- Identify the best matching source column for each target field.
- Mention if multiple columns must be combined (e.g., RN + LPN + CNA hours).
- Mention if no matching source column exists.
- Explain your reasoning in 1–2 lines per target field.
- Do not hallucinate or invent new columns.
- Keep response clean, readable, and ideally structured as bullet points.
"""

print("=== MAPPING PROMPT PREVIEW (first 800 chars) ===")
print(mapping_prompt[:800])


=== MAPPING PROMPT PREVIEW (first 800 chars) ===

You are a Data Mapping Agent.

Here is the source schema extracted from the dataset:
- PROVNUM: object
- PROVNAME: object
- CITY: object
- STATE: object
- COUNTY_NAME: object
- COUNTY_FIPS: int64
- CY_Qtr: object
- WorkDate: int64
- MDScensus: int64
- Hrs_RNDON: float64
- Hrs_RNDON_emp: float64
- Hrs_RNDON_ctr: float64
- Hrs_RNadmin: float64
- Hrs_RNadmin_emp: float64
- Hrs_RNadmin_ctr: float64
- Hrs_RN: float64
- Hrs_RN_emp: float64
- Hrs_RN_ctr: float64
- Hrs_LPNadmin: float64
- Hrs_LPNadmin_emp: float64
- Hrs_LPNadmin_ctr: float64
- Hrs_LPN: float64
- Hrs_LPN_emp: float64
- Hrs_LPN_ctr: float64
- Hrs_CNA: float64
- Hrs_CNA_emp: float64
- Hrs_CNA_ctr: float64
- Hrs_NAtrn: float64
- Hrs_NAtrn_emp: float64
- Hrs_NAtrn_ctr: float64
- Hrs_MedAide: float64
- Hrs_MedAide_emp: float64
- Hrs_Me


## 🟦 Step 8 — AI Mapping Agent (Using ADK + Gemini)

This step implements the **MappingAgent** using the Agent Development Kit (ADK).

The MappingAgent:
- Takes the source schema (bullet-list)
- Takes the target schema
- Uses an **LLM-powered ADK agent** (Gemini model)
- Returns a structured mapping from source → target fields
- Explains reasoning for each field


In [12]:
# Step 8 - Generic MappingAgent for any tabular dataset

import os
from kaggle_secrets import UserSecretsClient

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner

# 1️⃣ Load API key from Kaggle secrets
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("🔑 GOOGLE_API_KEY loaded from Kaggle Secrets")
except Exception as e:
    print("❌ ERROR: Please set GOOGLE_API_KEY in your Kaggle Secrets", e)

# 2️⃣ Define a GENERIC MappingAgent
mapping_agent = LlmAgent(
    name="mapping_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
You are a professional schema mapping assistant for tabular data.

You are ALWAYS given:
1) A SOURCE SCHEMA: a list of column names and data types from some tabular dataset.
2) A TARGET SCHEMA: a list of desired output fields.

Your job:
- For EACH target field, decide how it can be derived from the source schema.
- If it maps directly to a single source column, state that clearly.
- If it requires multiple source columns (e.g., sum of several numeric columns), list them and say what to do.
- If there is no reasonable source column, clearly say "NO MATCH".

Rules:
- Never invent or assume columns that are not in the source schema.
- Be concise but explicit: 1–2 lines of reasoning per target field.
- Keep the response structured and easy to read.
"""
)

print("✅ Generic MappingAgent created via ADK")

# 3️⃣ Create a runner
mapping_runner = InMemoryRunner(agent=mapping_agent)
print("🚀 InMemoryRunner created for MappingAgent")

# 4️⃣ Build a generic mapping prompt
mapping_prompt = f"""
You are mapping from an arbitrary SOURCE schema to a TARGET schema.

In this run, the source schema comes from a specific dataset (for example, nurse staffing),
but you must treat it generically as just "tabular data with columns".

SOURCE SCHEMA:
{source_schema_text}

TARGET SCHEMA:
{target_schema_text}

Please:
- For each TARGET field, map it to one or more SOURCE columns when possible.
- Indicate column combinations explicitly (e.g., sum of A + B + C).
- Mark missing mappings as NO MATCH.
- Provide a short reasoning for each mapping.
"""

print("🧾 Prepared generic mapping prompt. Calling MappingAgent...")

# 5️⃣ Call the agent (returns a LIST of events)
mapping_events = await mapping_runner.run_debug(mapping_prompt)

# 6️⃣ Extract the final text from events
mapping_response_text = ""

for event in mapping_events:
    if hasattr(event, "is_final_response") and event.is_final_response():
        if getattr(event, "content", None) and getattr(event.content, "parts", None):
            mapping_response_text = "".join(
                getattr(part, "text", "") for part in event.content.parts
            )
            break

print("=== 🧠 Generic Schema Mapping Output (MappingAgent) ===")
print(mapping_response_text)


🔑 GOOGLE_API_KEY loaded from Kaggle Secrets
✅ Generic MappingAgent created via ADK
🚀 InMemoryRunner created for MappingAgent
🧾 Prepared generic mapping prompt. Calling MappingAgent...

 ### Created new session: debug_session_id

User > 
You are mapping from an arbitrary SOURCE schema to a TARGET schema.

In this run, the source schema comes from a specific dataset (for example, nurse staffing),
but you must treat it generically as just "tabular data with columns".

SOURCE SCHEMA:
- PROVNUM: object
- PROVNAME: object
- CITY: object
- STATE: object
- COUNTY_NAME: object
- COUNTY_FIPS: int64
- CY_Qtr: object
- WorkDate: int64
- MDScensus: int64
- Hrs_RNDON: float64
- Hrs_RNDON_emp: float64
- Hrs_RNDON_ctr: float64
- Hrs_RNadmin: float64
- Hrs_RNadmin_emp: float64
- Hrs_RNadmin_ctr: float64
- Hrs_RN: float64
- Hrs_RN_emp: float64
- Hrs_RN_ctr: float64
- Hrs_LPNadmin: float64
- Hrs_LPNadmin_emp: float64
- Hrs_LPNadmin_ctr: float64
- Hrs_LPN: float64
- Hrs_LPN_emp: float64
- Hrs_LPN_ctr: flo

## 🟦 Step 9 — Review & Interpret MappingAgent Output

In this step, I examine the output generated by the ADK MappingAgent.

I review:
- Which source fields were mapped to each target field
- How the agent decided RN/LPN/CNA hour combinations
- Whether any target fields were missing or unmapped
- The quality of the reasoning and correctness

This completes the second agent in the multi-agent sequence:
**SchemaAgent → MappingAgent**

The final mapping text is displayed below for clarity and verification.


## 🟦 Step 10 — TransformAgent (AI-Generated Transformation Logic)

The MappingAgent tells me *which* columns map to the target schema.

Next, the TransformAgent generates **Python transformation code** that:

- computes rn_hours_total  
- computes lpn_hours_total  
- computes cna_hours_total  
- computes total_nursing_hours  
- creates the final cleaned dataframe

This demonstrates:

- **Sequential multi-agent flow**
- **LLM agent generating executable code**
- **Custom tool integration** (code executor)
- A core concept from the Capstone rubric


In [13]:
# Step 10 - Generic TransformAgent (generate pandas transformation code)

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner

# 1️⃣ Define a GENERIC TransformAgent
transform_agent = LlmAgent(
    name="transform_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
You are a Senior Data Engineer writing pandas ETL code for arbitrary tabular data.

You are given:
- A mapping description between SOURCE columns and TARGET fields.
- The source data is already loaded into a pandas DataFrame named `df`.
- The target schema is a list of final column names.

Your job:
1. Write Python (pandas) code that:
   - Starts from the existing `df` with the SOURCE columns.
   - Creates the TARGET columns using only the mapped source columns.
   - Performs any aggregations or combinations described in the mapping (e.g., sums).
   - Outputs a new DataFrame named `df_out` that has EXACTLY the target schema columns in order.

2. Do NOT invent columns that were not mentioned in the mapping.
3. Assume this code will be pasted into a cell where `df` already exists.
4. Return ONLY valid Python code (no explanation, no comments, no markdown).
"""
)

print("✅ Generic TransformAgent created via ADK")

# 2️⃣ Runner for TransformAgent
transform_runner = InMemoryRunner(agent=transform_agent)
print("🚀 InMemoryRunner created for TransformAgent")

# 3️⃣ Build prompt using the generic mapping result
transform_prompt = f"""
You are given this mapping between SOURCE columns and TARGET fields:

{mapping_response_text}

The target schema fields are:

{target_schema_text}

Generate the Python (pandas) transformation code according to your instructions.
Remember:
- Input DataFrame name: df
- Output DataFrame name: df_out
"""

print("🧾 Prepared generic transform prompt. Calling TransformAgent...")

# 4️⃣ Call the agent
transform_events = await transform_runner.run_debug(transform_prompt)

# 5️⃣ Extract final code text
transform_code_text = ""

for event in transform_events:
    if hasattr(event, "is_final_response") and event.is_final_response():
        if getattr(event, "content", None) and getattr(event.content, "parts", None):
            transform_code_text = "".join(
                getattr(part, "text", "") for part in event.content.parts
            )
            break

print("=== 🧮 Generated Generic Transformation Code (TransformAgent) ===")
print(transform_code_text)


✅ Generic TransformAgent created via ADK
🚀 InMemoryRunner created for TransformAgent
🧾 Prepared generic transform prompt. Calling TransformAgent...

 ### Created new session: debug_session_id

User > 
You are given this mapping between SOURCE columns and TARGET fields:

facility_id: PROVNUM - Direct mapping from the provider number.
facility_name: PROVNAME - Direct mapping from the provider name.
state: STATE - Direct mapping from the state column.
county_name: COUNTY_NAME - Direct mapping from the county name.
quarter: CY_Qtr - Direct mapping from the quarter column.
work_date: WorkDate - Direct mapping from the work date.
resident_census: MDScensus - Direct mapping from the medical/surgical census.
rn_hours_total: Hrs_RN - Direct mapping from the total RN hours.
lpn_hours_total: Hrs_LPN - Direct mapping from the total LPN hours.
cna_hours_total: Hrs_CNA - Direct mapping from the total CNA hours.
total_nursing_hours: sum of Hrs_RN + Hrs_LPN + Hrs_CNA + Hrs_MedAide + Hrs_NAtrn - Summin

## 🟦 Step 11 — ValidationAgent (Check Correctness & Quality)

Once transformation logic is suggested by the TransformAgent,
the ValidationAgent evaluates:

- whether required target fields are present
- whether data types match expectations
- whether hour totals are computed correctly
- if any fields are missing or incorrectly mapped

This completes the 3-agent chain:
**SchemaAgent → MappingAgent → TransformAgent → ValidationAgent**


In [14]:
# Step 11 - Generic ValidationAgent (validate transformation code)

validation_agent = LlmAgent(
    name="validation_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
You are a strict ETL Validation Agent for arbitrary tabular data.

You are given:
- Python (pandas) transformation code that starts from a DataFrame `df`
  and creates a new DataFrame `df_out`.
- The target schema: a list of required output columns.

Your job:
1. Check that the code defines `df_out` and that `df_out` includes ALL required target columns.
2. Check that the code ONLY uses source columns that could reasonably exist
   given the mapping description (if provided).
3. Identify:
   - Missing target columns
   - Suspicious or undefined column references
   - Obvious logic problems (e.g., using a field before defining it)

4. Return a structured validation report with:
   - PASS/FAIL for the overall transformation
   - A list of issues found (if any)
   - Concrete suggestions to fix them.

You do NOT execute the code; you are doing a static reasoning review.
"""
)

print("✅ Generic ValidationAgent created via ADK")

validation_runner = InMemoryRunner(agent=validation_agent)
print("🚀 InMemoryRunner created for ValidationAgent")

# Build validation prompt
validation_prompt = f"""
Here is the transformation code:

{transform_code_text}

The expected target schema (columns in df_out) is:

{target_schema_text}

Validate this code and produce a structured PASS/FAIL report as per your instructions.
"""

print("🧾 Prepared generic validation prompt. Calling ValidationAgent...")

# Call the agent
validation_events = await validation_runner.run_debug(validation_prompt)

# Extract final validation text
validation_output_text = ""

for event in validation_events:
    if hasattr(event, "is_final_response") and event.is_final_response():
        if getattr(event, "content", None) and getattr(event.content, "parts", None):
            validation_output_text = "".join(
                getattr(part, "text", "") for part in event.content.parts
            )
            break

print("=== ✅ Generic Validation Report (ValidationAgent) ===")
print(validation_output_text)


✅ Generic ValidationAgent created via ADK
🚀 InMemoryRunner created for ValidationAgent
🧾 Prepared generic validation prompt. Calling ValidationAgent...

 ### Created new session: debug_session_id

User > 
Here is the transformation code:

```python
df_out = df.rename(columns={
    'PROVNUM': 'facility_id',
    'PROVNAME': 'facility_name',
    'STATE': 'state',
    'COUNTY_NAME': 'county_name',
    'CY_Qtr': 'quarter',
    'WorkDate': 'work_date',
    'MDScensus': 'resident_census',
    'Hrs_RN': 'rn_hours_total',
    'Hrs_LPN': 'lpn_hours_total',
    'Hrs_CNA': 'cna_hours_total'
})

df_out['total_nursing_hours'] = df_out['rn_hours_total'].fillna(0) + \
                               df_out['lpn_hours_total'].fillna(0) + \
                               df_out['cna_hours_total'].fillna(0)

# Add other potential nursing hours columns if they exist in df
if 'Hrs_MedAide' in df.columns:
    df_out['total_nursing_hours'] += df['Hrs_MedAide'].fillna(0)
if 'Hrs_NAtrn' in df.columns:
    df_ou

### 🧪 Using ValidationAgent Feedback

The initial TransformAgent code:
- Included nurse-specific columns (`Hrs_MedAide`, `Hrs_NAtrn`) that were not in the target schema.
- Used redundant logic (loop + explicit column selection).

ValidationAgent correctly flagged this with a FAIL and suggested improvements.

Based on that feedback, I simplified the transformation:
- Only mapped columns from `df` are used.
- `total_nursing_hours` is computed as RN + LPN + CNA totals.
- The final `df_out` DataFrame contains exactly the target schema columns.


In [18]:
### Final cleaned transformation code (after ValidationAgent feedback)
import pandas as pd

# Start from the raw df and rename source columns to target-friendly names
df_out = df.rename(columns={
    'PROVNUM': 'facility_id',
    'PROVNAME': 'facility_name',
    'STATE': 'state',
    'COUNTY_NAME': 'county_name',
    'CY_Qtr': 'quarter',
    'WorkDate': 'work_date',
    'MDScensus': 'resident_census',
    'Hrs_RN': 'rn_hours_total',
    'Hrs_LPN': 'lpn_hours_total',
    'Hrs_CNA': 'cna_hours_total'
}).copy()

# Ensure hour fields exist and fill missing values with 0 before summing
for col in ['rn_hours_total', 'lpn_hours_total', 'cna_hours_total']:
    if col not in df_out.columns:
        df_out[col] = 0
    df_out[col] = df_out[col].fillna(0)

# Compute total nursing hours as the sum of RN, LPN and CNA hours
df_out['total_nursing_hours'] = (
    df_out['rn_hours_total'] +
    df_out['lpn_hours_total'] +
    df_out['cna_hours_total']
)

# Final output: only the target schema columns, in the desired order
df_out = df_out[
    [
        'facility_id',
        'facility_name',
        'state',
        'county_name',
        'quarter',
        'work_date',
        'resident_census',
        'rn_hours_total',
        'lpn_hours_total',
        'cna_hours_total',
        'total_nursing_hours'
    ]
]
print("df_out shape:", df_out.shape)
df_out.head()

df_out shape: (1325324, 11)


,facility_id,facility_name,state,county_name,quarter,work_date,resident_census,rn_hours_total,lpn_hours_total,cna_hours_total,total_nursing_hours
0,15009,"BURNS NURSING HOME, INC.",AL,Franklin,2024Q2,20240401,51,55.70,25.50,160.08,241.28
1,15009,"BURNS NURSING HOME, INC.",AL,Franklin,2024Q2,20240402,52,63.28,15.22,135.95,214.45
2,15009,"BURNS NURSING HOME, INC.",AL,Franklin,2024Q2,20240403,53,76.29,5.46,150.31,232.06
3,15009,"BURNS NURSING HOME, INC.",AL,Franklin,2024Q2,20240404,52,54.13,20.18,133.01,207.32
4,15009,"BURNS NURSING HOME, INC.",AL,Franklin,2024Q2,20240405,52,53.63,27.85,137.92,219.40


## 🟦 Step 12 — Multi-Agent Orchestrator (Mapping → Transform → Validation)

In this step, I connect the three agents into a single pipeline:

1. **MappingAgent**  
   - Maps the source schema to the target schema.

2. **TransformAgent**  
   - Generates pandas transformation code based on the mapping.

3. **ValidationAgent**  
   - Reviews the generated code and returns a PASS/FAIL-style report with issues and suggestions.

The orchestrator runs all three in sequence and returns a dictionary with:
- `mapping` — human-readable mapping output  
- `transform_code` — generated pandas code (first draft)  
- `validation` — validation report for that code  


In [19]:
# Step 12 — Orchestrator for the 3-Agent ETL Pipeline

import asyncio
import nest_asyncio
nest_asyncio.apply()

async def run_etl_orchestrator(source_schema_text: str, target_schema_text: str):
    """
    Runs the 3-agent ETL pipeline:
    1. MappingAgent  -> mapping_text
    2. TransformAgent -> transform_code (first draft)
    3. ValidationAgent -> validation_report
    """

    print("\n🚀 Starting ETL Orchestrator...\n")

    # ======================
    # 1️⃣ MAPPING AGENT
    # ======================
    print("▶️ Running MappingAgent...")

    mapping_prompt = f"""
Map the following SOURCE schema to the TARGET schema.

SOURCE SCHEMA:
{source_schema_text}

TARGET SCHEMA:
{target_schema_text}

Rules:
- For each TARGET field, map it to one or more SOURCE columns.
- If combining columns, explicitly state which ones.
- If no mapping exists, say NO MATCH.
- Do not invent columns that do not exist in the source schema.
"""

    mapping_events = await mapping_runner.run_debug(mapping_prompt)

    mapping_text = ""
    for event in mapping_events:
        if hasattr(event, "is_final_response") and event.is_final_response():
            if getattr(event, "content", None) and event.content.parts:
                mapping_text = "".join(getattr(p, "text", "") for p in event.content.parts)
                break

    print("✓ MappingAgent complete.\n")


    # ======================
    # 2️⃣ TRANSFORM AGENT
    # ======================
    print("▶️ Running TransformAgent...")

    transform_prompt = f"""
You are a Senior Data Engineer.

Here is the mapping between SOURCE columns and TARGET fields:

{mapping_text}

The target schema fields are:

{target_schema_text}

Generate Python (pandas) code that:
- Starts from a DataFrame named `df` with the SOURCE columns.
- Produces a DataFrame named `df_out` with exactly the TARGET columns.
- Uses only mapped source columns.
- Returns only valid Python code (no comments, no markdown).
"""

    transform_events = await transform_runner.run_debug(transform_prompt)

    transform_code = ""
    for event in transform_events:
        if hasattr(event, "is_final_response") and event.is_final_response():
            if getattr(event, "content", None) and event.content.parts:
                transform_code = "".join(getattr(p, "text", "") for p in event.content.parts)
                break

    print("✓ TransformAgent complete.\n")


    # ======================
    # 3️⃣ VALIDATION AGENT
    # ======================
    print("▶️ Running ValidationAgent...")

    validation_prompt = f"""
Here is the generated pandas transformation code:

{transform_code}

The expected target schema (columns in df_out) is:

{target_schema_text}

Validate this code statically and produce a structured PASS/FAIL report with:
- Issues (e.g., missing columns, suspicious references)
- Suggestions for how to fix them.
"""

    validation_events = await validation_runner.run_debug(validation_prompt)

    validation_report = ""
    for event in validation_events:
        if hasattr(event, "is_final_response") and event.is_final_response():
            if getattr(event, "content", None) and event.content.parts:
                validation_report = "".join(
                    getattr(p, "text", "") for p in event.content.parts
                )
                break

    print("✓ ValidationAgent complete.\n")
    print("🎉 ETL Orchestrator Finished!")

    # Return all outputs
    return {
        "mapping": mapping_text,
        "transform_code": transform_code,
        "validation": validation_report,
    }


# ⭐ Run the orchestrator once to capture outputs
loop = asyncio.get_event_loop()
pipeline_output = loop.run_until_complete(
    run_etl_orchestrator(source_schema_text, target_schema_text)
)

pipeline_output



🚀 Starting ETL Orchestrator...

▶️ Running MappingAgent...

 ### Continue session: debug_session_id

User > 
Map the following SOURCE schema to the TARGET schema.

SOURCE SCHEMA:
- PROVNUM: object
- PROVNAME: object
- CITY: object
- STATE: object
- COUNTY_NAME: object
- COUNTY_FIPS: int64
- CY_Qtr: object
- WorkDate: int64
- MDScensus: int64
- Hrs_RNDON: float64
- Hrs_RNDON_emp: float64
- Hrs_RNDON_ctr: float64
- Hrs_RNadmin: float64
- Hrs_RNadmin_emp: float64
- Hrs_RNadmin_ctr: float64
- Hrs_RN: float64
- Hrs_RN_emp: float64
- Hrs_RN_ctr: float64
- Hrs_LPNadmin: float64
- Hrs_LPNadmin_emp: float64
- Hrs_LPNadmin_ctr: float64
- Hrs_LPN: float64
- Hrs_LPN_emp: float64
- Hrs_LPN_ctr: float64
- Hrs_CNA: float64
- Hrs_CNA_emp: float64
- Hrs_CNA_ctr: float64
- Hrs_NAtrn: float64
- Hrs_NAtrn_emp: float64
- Hrs_NAtrn_ctr: float64
- Hrs_MedAide: float64
- Hrs_MedAide_emp: float64
- Hrs_MedAide_ctr: float64

TARGET SCHEMA:
facility_id
facility_name
state
county_name
quarter
work_date
resident_

{'mapping': 'facility_id: PROVNUM - Direct mapping from the provider number.\nfacility_name: PROVNAME - Direct mapping from the provider name.\nstate: STATE - Direct mapping from the state column.\ncounty_name: COUNTY_NAME - Direct mapping from the county name.\nquarter: CY_Qtr - Direct mapping from the quarter column.\nwork_date: WorkDate - Direct mapping from the work date.\nresident_census: MDScensus - Direct mapping from the medical/surgical census.\nrn_hours_total: Hrs_RN - Direct mapping from the total RN hours.\nlpn_hours_total: Hrs_LPN - Direct mapping from the total LPN hours.\ncna_hours_total: Hrs_CNA - Direct mapping from the total CNA hours.\ntotal_nursing_hours: sum of Hrs_RN + Hrs_LPN + Hrs_CNA + Hrs_MedAide + Hrs_NAtrn - Summing up all direct nursing staff hours.',
 'transform_code': "```python\ndf_out = df.rename(columns={\n    'PROVNUM': 'facility_id',\n    'PROVNAME': 'facility_name',\n    'STATE': 'state',\n    'COUNTY_NAME': 'county_name',\n    'CY_Qtr': 'quarter',\

## 🟦 Step 13 — Final Cleaned Transformation Code (Human-in-the-Loop)

Based on the ValidationAgent feedback, I simplified and corrected the
pandas transformation code.

Key improvements:
- Removed extra nurse-specific columns that were not part of the target schema.
- Removed redundant logic (double-checking columns and re-selecting).
- Ensured that `df_out` has exactly the target columns, in the correct order.
- Explicitly computes `total_nursing_hours` as RN + LPN + CNA hours.

This cleaned code is what I consider the **final ETL step** for this dataset.
The DocumentationAgent in the next step will document **this** version of the code.


In [20]:
# Step 13 — Final cleaned transformation code (after ValidationAgent feedback)

clean_transform_code = """
import pandas as pd

# Start from the raw df and rename source columns to target-friendly names
df_out = df.rename(columns={
    'PROVNUM': 'facility_id',
    'PROVNAME': 'facility_name',
    'STATE': 'state',
    'COUNTY_NAME': 'county_name',
    'CY_Qtr': 'quarter',
    'WorkDate': 'work_date',
    'MDScensus': 'resident_census',
    'Hrs_RN': 'rn_hours_total',
    'Hrs_LPN': 'lpn_hours_total',
    'Hrs_CNA': 'cna_hours_total'
}).copy()

# Ensure hour fields exist and fill missing values with 0 before summing
for col in ['rn_hours_total', 'lpn_hours_total', 'cna_hours_total']:
    if col not in df_out.columns:
        df_out[col] = 0
    df_out[col] = df_out[col].fillna(0)

# Compute total nursing hours as the sum of RN, LPN and CNA hours
df_out['total_nursing_hours'] = (
    df_out['rn_hours_total'] +
    df_out['lpn_hours_total'] +
    df_out['cna_hours_total']
)

# Final output: only the target schema columns, in the desired order
df_out = df_out[
    [
        'facility_id',
        'facility_name',
        'state',
        'county_name',
        'quarter',
        'work_date',
        'resident_census',
        'rn_hours_total',
        'lpn_hours_total',
        'cna_hours_total',
        'total_nursing_hours'
    ]
]
"""

print("=== 🧮 Final Cleaned Transformation Code ===")
print(clean_transform_code)


=== 🧮 Final Cleaned Transformation Code ===

import pandas as pd

# Start from the raw df and rename source columns to target-friendly names
df_out = df.rename(columns={
    'PROVNUM': 'facility_id',
    'PROVNAME': 'facility_name',
    'STATE': 'state',
    'COUNTY_NAME': 'county_name',
    'CY_Qtr': 'quarter',
    'WorkDate': 'work_date',
    'MDScensus': 'resident_census',
    'Hrs_RN': 'rn_hours_total',
    'Hrs_LPN': 'lpn_hours_total',
    'Hrs_CNA': 'cna_hours_total'
}).copy()

# Ensure hour fields exist and fill missing values with 0 before summing
for col in ['rn_hours_total', 'lpn_hours_total', 'cna_hours_total']:
    if col not in df_out.columns:
        df_out[col] = 0
    df_out[col] = df_out[col].fillna(0)

# Compute total nursing hours as the sum of RN, LPN and CNA hours
df_out['total_nursing_hours'] = (
    df_out['rn_hours_total'] +
    df_out['lpn_hours_total'] +
    df_out['cna_hours_total']
)

# Final output: only the target schema columns, in the desired order
df_ou

## 🟦 Step 14 — DocumentationAgent (Auto-Generated Markdown Docs)

Finally, I add a `DocumentationAgent` that generates human-readable documentation
for this ETL pipeline.

It uses:
- The schema mapping (MappingAgent output)
- The final cleaned transformation code
- The validation report (ValidationAgent output)
- The target schema

The output is a Markdown document that includes:
- A project overview
- A data dictionary for the target schema
- A source-to-target mapping summary
- A description of the multi-agent pipeline
- A short validation summary



In [21]:
# Step 14 — DocumentationAgent (generate Markdown documentation)

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner

# 1️⃣ Extract from pipeline_output
mapping_text = pipeline_output["mapping"]
validation_report = pipeline_output["validation"]

# 2️⃣ Use the FINAL CLEANED code, not the raw TransformAgent draft
transform_code_for_docs = clean_transform_code

# 3️⃣ Define DocumentationAgent
documentation_agent = LlmAgent(
    name="documentation_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
You are a technical documentation writer for data engineering pipelines.

You are given:
- A description of how SOURCE columns map to TARGET fields (mapping_text).
- The final, cleaned pandas transformation code (transform_code).
- A validation report describing checks and issues (validation_report).
- The target schema (column list).

Your job:
Generate a clear Markdown document that includes:

1) **Project Overview**
   - 2–3 sentences explaining that this is a generic ETL copilot that:
     - maps any tabular source schema to a target schema,
     - generates pandas code,
     - validates it.
   - Mention that this run uses a nurse staffing dataset as an example.

2) **Target Schema – Data Dictionary**
   - A bullet list where each target column has a short description.
   - If exact business definitions are unknown, infer a reasonable one from the names.

3) **Source-to-Target Mapping Summary**
   - A Markdown table with columns:
     - Target Field
     - Source Column(s)
     - Notes / Logic
   - Populate it by interpreting the mapping_text.

4) **ETL Pipeline Overview**
   - A short subsection explaining the three main agents:
     - MappingAgent: maps source schema to target schema
     - TransformAgent: generates pandas transformation code
     - ValidationAgent: reviews the code and reports PASS/FAIL
   - Mention that this forms a sequential multi-agent pipeline.

5) **Validation Summary**
   - Briefly summarize whether the validation report shows PASS or FAIL.
   - List the main issues or confirmations in 2–3 bullets.

Output:
- The final answer must be valid Markdown (no extra backticks wrapping the entire doc).
"""
)

print("✅ DocumentationAgent created via ADK")

# 4️⃣ Runner for DocumentationAgent
documentation_runner = InMemoryRunner(agent=documentation_agent)
print("🚀 InMemoryRunner created for DocumentationAgent")

# 5️⃣ Build prompt
documentation_prompt = f"""
Here is the schema mapping:

{mapping_text}

Here is the final cleaned pandas transformation code:

{transform_code_for_docs}

Here is the validation report:

{validation_report}

The target schema (final columns) is:

{target_schema_text}

Please generate the Markdown documentation as per your instructions.
"""

print("🧾 Prepared documentation prompt. Calling DocumentationAgent...")

# 6️⃣ Call DocumentationAgent
documentation_events = await documentation_runner.run_debug(documentation_prompt)

documentation_markdown = ""

for event in documentation_events:
    if hasattr(event, "is_final_response") and event.is_final_response():
        if getattr(event, "content", None) and getattr(event.content, "parts", None):
            documentation_markdown = "".join(
                getattr(part, "text", "") for part in event.content.parts
            )
            break

print("=== 📄 Auto-Generated Documentation (DocumentationAgent) ===")
print(documentation_markdown)


✅ DocumentationAgent created via ADK
🚀 InMemoryRunner created for DocumentationAgent
🧾 Prepared documentation prompt. Calling DocumentationAgent...

 ### Created new session: debug_session_id

User > 
Here is the schema mapping:

facility_id: PROVNUM - Direct mapping from the provider number.
facility_name: PROVNAME - Direct mapping from the provider name.
state: STATE - Direct mapping from the state column.
county_name: COUNTY_NAME - Direct mapping from the county name.
quarter: CY_Qtr - Direct mapping from the quarter column.
work_date: WorkDate - Direct mapping from the work date.
resident_census: MDScensus - Direct mapping from the medical/surgical census.
rn_hours_total: Hrs_RN - Direct mapping from the total RN hours.
lpn_hours_total: Hrs_LPN - Direct mapping from the total LPN hours.
cna_hours_total: Hrs_CNA - Direct mapping from the total CNA hours.
total_nursing_hours: sum of Hrs_RN + Hrs_LPN + Hrs_CNA + Hrs_MedAide + Hrs_NAtrn - Summing up all direct nursing staff hours.

Her

## 🟦 Step 15 — Observability (Logging, Tracing, Metrics)

A production-grade multi-agent system requires strong observability.

In this notebook, observability is demonstrated through:

### 🔹 Logging
Each agent prints:
- Start and end of execution
- Structured summaries of outputs
- Validation PASS/FAIL status

### 🔹 Tracing
The orchestrator prints:
- "Running MappingAgent…"
- "Running TransformAgent…"
- "Running ValidationAgent…"

This provides a clear execution trace of the pipeline.

### 🔹 Lightweight Metrics
Examples include:
- Length of mapping text
- Lines of transformation code
- PASS/FAIL from validation

This satisfies the Capstone requirement for **Logging, Tracing, and basic Metrics**.


In [22]:
def log_agent_event(agent_name, event_type, details=""):
    print(f"[{agent_name}] {event_type}: {details}")

def count_lines(text: str):
    return len(text.split("\n"))

log_agent_event("MappingAgent", "OUTPUT_LENGTH_CHARS", str(len(pipeline_output["mapping"])))
log_agent_event("TransformAgent", "LINES_GENERATED", str(count_lines(pipeline_output["transform_code"])))
log_agent_event(
    "ValidationAgent",
    "STATUS",
    "PASS" if "PASS" in pipeline_output["validation"] else "FAIL"
)


[MappingAgent] OUTPUT_LENGTH_CHARS: 765
[TransformAgent] LINES_GENERATED: 44
[ValidationAgent] STATUS: PASS


## 🟦 Step 14 — Final Capstone Summary

This notebook implements a complete **Multi-Agent ETL Copilot** using the Google Agent Development Kit (ADK).

### ✅ Core Capstone Features

**Multi-Agent System (4 agents):**
- MappingAgent — schema mapping
- TransformAgent — pandas code generation
- ValidationAgent — static code validation
- DocumentationAgent — Markdown documentation

**Tools:**
- Custom schema extraction (Python)
- ADK `LlmAgent` + `InMemoryRunner`
- Gemini models for reasoning and generation

**Context Engineering:**
- Compact source schema representation
- Explicit target schema text
- Structured prompts for mapping, transformation, validation, and documentation

**Sessions & Memory:**
- In-memory orchestration using ADK
- Data passed explicitly from agent to agent

**Observability:**
- Console logs for each agent’s start, end, and outputs
- Sequential trace from Mapping → Transform → Validation → Documentation

**A2A Chaining:**
- MappingAgent → TransformAgent → ValidationAgent → DocumentationAgent

---

### 🎯 Outcome

The system:

- Works with **any tabular dataset** where a target schema is defined.
- Automates:
  - schema mapping,
  - code generation,
  - validation,
  - pipeline documentation.
- Demonstrates real-world **enterprise ETL automation** using multi-agent GenAI.
